Generate a set of random string sequences that follow a specified format, and add a few anomalies. We're gonna start by writing a function that creates strings of the following format: CEBF0ZPQ ([4 letters A-F][1 digit 0–2][3 letters QWOPZXML]), and generate 25K sequences of this format.

In [2]:
import random
import pandas as pd
first_letters =  'ABCDEF'
second_numbers = '120'
last_letters = 'QWOPZXML'
# returns a string of the following format: [4 letters A-F][1 digit 0-2][3 letters QWOPZXML]
def get_random_string():
    str1 = ''.join(random.choice(first_letters) for i in range(4))
    str2 = random.choice(second_numbers)
    str3 = ''.join(random.choice(last_letters) for i in range(3))
    return str1+str2+str3
# get 25,000 sequences of this format
random_sequences = [get_random_string() for i in range(25000)]
#this will return string according to the following format
# ['CBCA2QOM', 'FBEF0WZW', 'DBFB2ZML', 'BFCB2WXO']
# add some anomalies to our list
random_sequences.extend(['XYDC2DCA', 'TXSX1ABC','RNIU4XRE','AABDXUEI','SDRAC5RF'])
#save this to a dataframe
seqs_ds = pd.DataFrame(random_sequences)

Encode the string sequences into numbers and scale them

In [3]:
import numpy as np
#Build the char index that we will use to encode seqs to numbers 
#(this char index was written by Jason Brownlee from Machine Learning Mastery)
char_index = '0abcdefghijklmnopqrstuvwxyz'
char_index +='ABCDEFGHIJKLMNOPQRSTUVWXYZ'
char_index += '123456789'
char_index += '().,-/+=&$?@#!*:;_[]|%⸏{}\"\'' + ' ' +'\\'
char_to_int = dict((c, i) for i, c in enumerate(char_index))
int_to_char = dict((i, c) for i, c in enumerate(char_index))
from keras.preprocessing.sequence import pad_sequences
#function that convert a char seqs to numbers seqs 
#(it does a little more but lets leave it for now)
def encode_sequence_list(seqs, feat_n=0):
    encoded_seqs = []
    for seq in seqs:
        encoded_seq = [char_to_int[c] for c in seq]
        encoded_seqs.append(encoded_seq)
    if(feat_n > 0):
        encoded_seqs.append(np.zeros(feat_n))
    return pad_sequences(encoded_seqs, padding='post')

def decode_sequence_list(seqs):
    decoded_seqs = []
    for seq in seqs:
        decoded_seq = [int_to_char[i] for i in seq]
        decoded_seqs.append(decoded_seq)
    return decoded_seqs
    
# Using the char_index, the encode_sequence_list function
# will turn a string like this EBCA0OXO 
#to an array like this [29 32 27 27  0 42 42 38]

# encode each string seq to an integer array [[1],[5],[67]], [[45],[76],[7]
encoded_seqs = encode_sequence_list(random_sequences)
# mix everything up
np.random.shuffle(encoded_seqs)
encoded_seqs.shape

(25005, 8)

In [4]:
print(random_sequences[10])
print(encoded_seqs[10])

EAFE2OLL
[27 28 32 31 53 43 49 49]


Now we have an array of the following shape as every string sequence has 8 characters, each of which is encoded as a number which we will treat as a column. Finally, before feeding the data to the autoencoder I'm going to scale the data using a MinMaxScaler, and split it into a training and test set. Proper scaling can often significantly improve the performance of NNs so it is important to experiment with more than one method.

In [5]:
#Scale our data using a MinMaxScaler that will scale 
#each number so that it will be between 0 and 1
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler()
scaled_seqs = scaler.fit_transform(encoded_seqs)
#Create a test and train sets of our data
X_train = scaled_seqs[:20000]
X_test = scaled_seqs[20000:]

Design, fit and tune the autoencoder.
As mentioned earlier, there is more than one way to design an autoencoder. It is usually based on small hidden layers wrapped with larger layers (this is what creates the encoding-decoding effect). I have made a few tuning sessions in order to determine the best params to use here as different kinds of data usually lend themselves to very different best-performance parameters.

In [6]:
from keras.models import Model, load_model
from keras.layers import Input, Dense, Dropout
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

input_dim = X_train.shape[1] # the # features
encoding_dim = 8 # first layer
hidden_dim = int(encoding_dim / 2) #hideen layer

nb_epoch = 30
batch_size = 128
learning_rate = 0.1

input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh", activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(hidden_dim, activation="relu")(encoder)
decoder = Dense(encoding_dim, activation='relu')(encoder)
decoder = Dense(input_dim, activation='tanh')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
#create callback
filepath = 'my_best_model.hdf5'
checkpointer = ModelCheckpoint(filepath=filepath,                             
                             verbose=0, 
                             save_best_only=True)

tensorboard = TensorBoard(log_dir='/tmp/autoencoder', histogram_freq=0,
                          write_graph=True,
                          write_images=True)

history = autoencoder.fit(X_train, X_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test, X_test),
                    verbose=1,
                    callbacks=[checkpointer, tensorboard]).history

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-05-29 12:06:47.376619: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-29 12:06:47.376747: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-05-29 12:06:47.739846: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-05-29 12:06:47.739856: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-05-29 12:06:47.740190: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-05-29 12:06:47.906441: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (register

Epoch 1/30


2022-05-29 12:06:54.289146: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


 25/157 [===>..........................] - ETA: 0s - loss: 0.2933 - accuracy: 0.0468

2022-05-29 12:06:56.255865: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-05-29 12:06:56.255874: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-05-29 12:06:56.263561: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-05-29 12:06:56.267199: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-05-29 12:06:56.270863: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /tmp/autoencoder/train/plugins/profile/2022_05_29_12_06_56
2022-05-29 12:06:56.271746: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to /tmp/autoencoder/train/plugins/profile/2022_05_29_12_06_56/Toms-MacBook-Air.local.trace.json.gz
2022-05-29 12:06:56.274087: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /tmp/autoencoder/train/plugins/profile/2022_

157/157 [==============================] - 10s 9ms/step - loss: 0.2238 - accuracy: 0.0556 - val_loss: 0.0581 - val_accuracy: 0.0507


2022-05-29 12:06:57.551808: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 2/30
157/157 [==============================] - 1s 7ms/step - loss: 0.0481 - accuracy: 0.4326 - val_loss: 0.0386 - val_accuracy: 0.2188
Epoch 3/30
157/157 [==============================] - 1s 7ms/step - loss: 0.0377 - accuracy: 0.2202 - val_loss: 0.0358 - val_accuracy: 0.2304
Epoch 4/30
157/157 [==============================] - 1s 8ms/step - loss: 0.0325 - accuracy: 0.3423 - val_loss: 0.0226 - val_accuracy: 0.6420
Epoch 5/30
157/157 [==============================] - 1s 8ms/step - loss: 0.0199 - accuracy: 0.6413 - val_loss: 0.0154 - val_accuracy: 0.6420
Epoch 6/30
157/157 [==============================] - 1s 8ms/step - loss: 0.0151 - accuracy: 0.6363 - val_loss: 0.0142 - val_accuracy: 0.6420
Epoch 7/30
157/157 [==============================] - 1s 8ms/step - loss: 0.0139 - accuracy: 0.6420 - val_loss: 0.0133 - val_accuracy: 0.6464
Epoch 8/30
157/157 [==============================] - 1s 8ms/step - loss: 0.0131 - accuracy: 0.6457 - val_loss: 0.0124 - val_accuracy: 0.6671
Epoch 

In [ ]:
from matplotlib import pyplot as plt 
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['loss', 'val_loss'])
plt.show(block=False)

In [10]:
autoencoder = load_model(filepath)
print(f"Min Loss:{np.min(history['loss'])}")

Min Loss:0.003943485673516989


Calculate the Error and Find the Anomalies!
Now, we feed the data again as a whole to the autoencoder and check the error term on each sample. Recall that seqs_ds is a pandas DataFrame that holds the actual string sequences. Line #2 encodes each string, and line #4 scales it. Then, I use the predict() method to get the reconstructed inputs of the strings stored in seqs_ds. Finally, I get the error term for each data point by calculating the “distance” between the input data point (or the actual data point) and the output that was reconstructed by the autoencoder:
`mse = np.mean(np.power(actual_data - reconstructed_data, 2), axis=1)`

In [11]:
#encode all the data
encoded_seqs = encode_sequence_list(seqs_ds.iloc[:,0])
#scale it
scaled_data = MinMaxScaler().fit_transform(encoded_seqs)
#predict it
predicted = autoencoder.predict(scaled_data)
#get the error term
mse = np.mean(np.power(scaled_data - predicted, 2), axis=1)
#now add them to our data frame
seqs_ds['MSE'] = mse
display(seqs_ds)

2022-05-29 12:15:10.277559: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


,0,MSE
0,CEEF0OPL,0.003283
1,ADEB0XWO,0.002919
2,CFAA1XOW,0.004977
3,FEAE0PWW,0.005148
4,EFBE2XLP,0.003786
...,...,...
25000,XYDC2DCA,0.209306
25001,TXSX1ABC,0.356510
25002,RNIU4XRE,0.161482
25003,AABDXUEI,0.004716


After we store the error term in the data frame, we can see how well each input data was constructed by our autoencoder.

How do we find the anomalies?
Well, the first thing we need to do is decide what is our threshold, and that usually depends on our data and domain knowledge. Some will say that an anomaly is a data point that has an error term that is higher than 95% of our data, for example. That would be an appropriate threshold if we expect that 5% of our data will be anomalous. However, recall that we injected 5 anomalies to a list of 25,000 perfectly formatted sequences, which means that only 0.02% of our data is anomalous, so we want to set our threshold as higher than 99.98% of our data (or the 0.9998 percentile). So first let's find this threshold:

In [12]:
mse_threshold = np.quantile(seqs_ds["MSE"], 0.9998)
print(f"MSE 0.9998 threshold:{mse_threshold}")

MSE 0.9998 threshold:0.009507248666678644


Next, I will add an MSE_Outlier column to the data set and set it to 1 when the error term crosses this threshold.

In [13]:
seqs_ds['MSE_Outlier'] = 0
seqs_ds.loc[seqs_ds['MSE'] > mse_threshold, 'MSE_Outlier'] = 1

And now all we have to do is check how many outliers do we have and whether these outliers are the ones we injected and mixed in the data
['XYDC2DCA', 'TXSX1ABC','RNIU4XRE','AABDXUEI','SDRAC5RF']. So let's see how many outliers we have and whether they are the ones we injected.

In [14]:
print(f"NUm of MSE Outlier:{seqs_ds['MSE_Outlier'].sum()}")
seqs_ds.iloc[25000:]

NUm of MSE Outlier:6


,0,MSE,MSE_Outlier
25000,XYDC2DCA,0.209306,1
25001,TXSX1ABC,0.356510,1
25002,RNIU4XRE,0.161482,1
25003,AABDXUEI,0.004716,0
25004,SDRAC5RF,0.147246,1
